In [1]:
from chase.base import *
from chase.utils import *
from chase.fit import *
from mypy.explib.frey2014 import frey2014
pd.set_option('display.max_colwidth', 100)

In [ ]:
arr = []
for sid in frey2014.subjects():
    print sid
    data = frey2014.subject_fitdata(sid)
    for d in data:
        arr.append([sid, d['gid'], d['group'], d['data'][0,0], d['data'][0,1]])
        
    df = pd.DataFrame(arr, columns=['subject', 'problem', 'group', 'choice', 'samplesize'])
df.to_csv('frey_data.csv')

In [2]:
df = frey2014.load_data()

In [35]:
df.to_csv('/Users/markant/Dropbox/SequentialSamplingDFE/frey_trial_data.csv')

In [2]:
data = pd.read_csv('data/frey_data.csv', index_col=0)
problems = {gid: frey2014.get_options(gid) for gid in data.problem.unique()}
arr = []
for i, row in data.iterrows():
    arr.append(row['problem'].split('G')[0])
data['session'] = arr
data.head(10)

,subject,problem,group,choice,samplesize,session
0,1,S1G1,0,0,12,S1
1,1,S2G1,0,1,10,S2
2,1,S3G1,0,0,10,S3
3,1,S4G1,0,0,10,S4
4,1,S5G1,0,0,10,S5
5,1,S6G1,0,1,10,S6
6,1,S7G1,0,0,10,S7
7,1,S8G1,0,0,12,S8
8,1,S9G1,0,0,10,S9
9,1,S10G1,0,0,10,S10


# Fit individual subjects

In [9]:
# a list of possible free parameters and their ranges
# and starting point (optional)
PARS = {'theta': [1, 50],
        'p_stay': [0, 1, .5],
        'tau': [0, 1, .5],
        'prelec_gamma': [0, 5, 1.],
        'prelec_elevation': [0, 5, 1.],
        'pow_gain': [0., 20., 1.],
        'w_loss': [0., np.inf, 1.],
        'mu': [0., np.inf, 20.],
        'sc': [0., np.inf, 1.],
        'p_stop': [0, 1, .5]}

# parameters that are set to fixed values 
FIXED = {'c': 0.5,
         'theta': 30}

N_ITER = 3
OUTDIR = 'chase_fitresults_frey_session'

# a list of different parameter combinations that will
# be fit
PARSETS_GEOM = [#['p_stop', 'p_stay', 'tau'],
                #['p_stop', 'p_stay', 'tau', 'prelec_gamma'],
                ['p_stop', 'p_stay', 'tau', 'prelec_gamma', 'prelec_elevation'],
                #['p_stop', 'p_stay', 'tau', 'pow_gain'],
                ['p_stop', 'p_stay', 'tau', 'pow_gain', 'w_loss'],
               ]

PARSETS_NORMAL = [#['mu', 'sc', 'p_stay', 'tau'],
                  #['mu', 'sc', 'p_stay', 'tau', 'prelec_gamma'],
                  ['mu', 'sc', 'p_stay', 'tau', 'prelec_gamma', 'prelec_elevation'],
                  #['mu', 'sc', 'p_stay', 'tau', 'pow_gain'],
                  ['mu', 'sc', 'p_stay', 'tau', 'pow_gain', 'w_loss'],
                  ]

### Geometric

In [10]:
for sid in data.session.unique():

    SIM_ID = 'frey_individual_planned_geom_session=%s' % sid

    for parset in PARSETS_GEOM:

        fitting = {p: PARS[p] for p in parset}

        # initialize the model
        m = CHASEAlternateStoppingModel(drift='cpt',
                                        startdist='laplace',
                                        stoprule='geometric',
                                        problems=problems)

        # fit
        results = fit_mlh(m, problems, data[data.session==sid], 
                          SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR,
                          method='Powell')

        print results.sort('nllh').head(1)

frey_individual_planned_geom_session=S1(p_stay,p_stop,prelec_elevation,prelec_gamma,tau|c=0.5,theta=30)
0/3
theta: 30.0
iteration                    0
success                   True
nllh                  1390.575
k                            5
N                          280
bic                   2809.323
theta                       30
p_stay               0.6658153
p_stop              0.03567396
prelec_elevation      1.705123
prelec_gamma          1.092343
tau                   0.490592
Name: 0, dtype: object
1/3
theta: 30.0
iteration                    1
success                   True
nllh                  1390.575
k                            5
N                          280
bic                   2809.323
theta                       30
p_stay               0.6658153
p_stop              0.03567396
prelec_elevation      1.705123
prelec_gamma          1.092343
tau                   0.490592
Name: 1, dtype: object
2/3
theta: 30.0
iteration                    2
success                   T

### Truncated normal

In [11]:
for sid in data.session.unique():

    SIM_ID = 'frey_individual_planned_session=%s' % sid

    for parset in PARSETS_NORMAL:

        fitting = {p: PARS[p] for p in parset}
        fitting['mu'][2] = data[data.session==sid].samplesize.mean()
        
        # initialize the model
        m = CHASEAlternateStoppingModel(drift='cpt',
                                        startdist='laplace',
                                        stoprule='truncatednormal',
                                        problems=problems)

        # fit
        results = fit_mlh(m, problems, data[data.session==sid], 
                          SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR,
                          method='Powell')

        print results.sort('nllh').head(1)

frey_individual_planned_session=S1(mu,p_stay,prelec_elevation,prelec_gamma,sc,tau|c=0.5,theta=30)
0/3
theta: 30.0
iteration                     0
success                    True
nllh                   1394.348
k                             6
N                           280
bic                    2822.504
theta                        30
mu                  0.001042674
p_stay                0.7778684
prelec_elevation       1.746847
prelec_gamma           1.234247
sc                     34.31454
tau                    0.452512
Name: 0, dtype: object
1/3
theta: 30.0
iteration                     1
success                    True
nllh                   1394.348
k                             6
N                           280
bic                    2822.504
theta                        30
mu                  0.001042674
p_stay                0.7778684
prelec_elevation       1.746847
prelec_gamma           1.234247
sc                     34.31454
tau                    0.452512
Name: 1, dtype:

In [17]:
sid = 110

In [18]:
allresults = pd.DataFrame([], columns=['sim_id', 'k', 'N', 'nllh', 'bic'])

for parset in PARSETS_GEOM:
    fitting = {p: PARS[p] for p in parset}
    b = best_result('frey_individual_planned_geom_subj=%s' % sid, FIXED, fitting, outdir=OUTDIR, nopars=True)
    if b is None:
        print 'No success'
    else:
        allresults.loc[allresults.shape[0]] = b

for parset in PARSETS_NORMAL:
    fitting = {p: PARS[p] for p in parset}
    b = best_result('frey_individual_planned_subj=%s' % sid, FIXED, fitting, outdir=OUTDIR, nopars=True)
    if b is None:
        print 'No success'
    else:
        allresults.loc[allresults.shape[0]] = b

allresults.sort('bic')

,sim_id,k,N,nllh,bic
3,"frey_individual_planned_subj=110(mu,p_stay,sc,tau|c=0.5,theta=30)",4,84,233.800803,485.324874
4,"frey_individual_planned_subj=110(mu,p_stay,prelec_elevation,prelec_gamma,sc,tau|c=0.5,theta=30)",6,84,230.480376,487.545653
5,"frey_individual_planned_subj=110(mu,p_stay,pow_gain,sc,tau,w_loss|c=0.5,theta=30)",6,84,231.069884,488.724668
0,"frey_individual_planned_geom_subj=110(p_stay,p_stop,tau|c=0.5,theta=30)",3,84,1990.051243,3993.394936
1,"frey_individual_planned_geom_subj=110(p_stay,p_stop,prelec_elevation,prelec_gamma,tau|c=0.5,thet...",5,84,1986.719423,3995.592931
2,"frey_individual_planned_geom_subj=110(p_stay,p_stop,pow_gain,tau,w_loss|c=0.5,theta=30)",5,84,1987.319867,3996.793818
